# 🎨 Z-Image Turbo Generator
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Z-Image-jupyter/blob/main/Z_Image_Turbo_jupyter.ipynb)

In [ ]:
%%capture
!git clone -q https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!pip install -q -r requirements.txt
!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors -d /content/ComfyUI/models/diffusion_models -o z-image-turbo-fp8-e4m3fn.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors -d /content/ComfyUI/models/clip -o qwen_3_4b.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors -d /content/ComfyUI/models/vae -o ae.safetensors
print("✅ Installation complete!")

In [ ]:
%%capture
%cd /content/ComfyUI
import os, random, time, torch
import numpy as np
from PIL import Image
from nodes import NODE_CLASS_MAPPINGS

UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
CLIPLoader = NODE_CLASS_MAPPINGS["CLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet = UNETLoader.load_unet("z-image-turbo-fp8-e4m3fn.safetensors", "fp8_e4m3fn_fast")[0]
    clip = CLIPLoader.load_clip("qwen_3_4b.safetensors", type="lumina2")[0]
    vae = VAELoader.load_vae("ae.safetensors")[0]

@torch.inference_mode()
def generate(positive_prompt, negative_prompt="blurry ugly bad", width=1024, height=1024, 
             steps=9, cfg=1.0, seed=0, sampler_name="euler", scheduler="simple", denoise=1.0, batch_size=1):
    tmp_dir = "/content/ComfyUI/output"
    os.makedirs(tmp_dir, exist_ok=True)
    
    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)
    
    positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    negative = CLIPTextEncode.encode(clip, negative_prompt)[0]
    latent_image = EmptyLatentImage.generate(width, height, batch_size=batch_size)[0]
    samples = KSampler.sample(unet, seed, steps, cfg, sampler_name, scheduler, positive, negative, latent_image, denoise=denoise)[0]
    decoded = VAEDecode.decode(vae, samples)[0].detach()
    output_path = f"{tmp_dir}/z_image_{seed}.png"
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(output_path)
    return output_path, seed

print("✅ Models loaded successfully!")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

style = {'description_width': '120px'}
layout = widgets.Layout(width='600px')

prompt_input = widgets.Textarea(
    value='一位男士和他的贵宾犬穿着配套的服装参加狗狗秀，室内灯光，背景中有观众。',
    description='Prompt:',
    layout=widgets.Layout(width='600px', height='80px'),
    style=style
)

negative_input = widgets.Textarea(
    value='blurry ugly bad',
    description='Negative:',
    layout=widgets.Layout(width='600px', height='60px'),
    style=style
)

width_slider = widgets.IntSlider(value=1024, min=512, max=2048, step=64, description='Width:', style=style, layout=layout)
height_slider = widgets.IntSlider(value=1024, min=512, max=2048, step=64, description='Height:', style=style, layout=layout)
steps_slider = widgets.IntSlider(value=9, min=1, max=50, step=1, description='Steps:', style=style, layout=layout)
cfg_slider = widgets.FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='CFG Scale:', style=style, layout=layout)
seed_input = widgets.IntText(value=0, description='Seed (0=random):', style=style, layout=layout)

sampler_dropdown = widgets.Dropdown(
    options=['euler', 'euler_ancestral', 'heun', 'dpm_2', 'dpm_2_ancestral', 'lms', 'dpm_fast', 'dpm_adaptive'],
    value='euler',
    description='Sampler:',
    style=style,
    layout=layout
)

scheduler_dropdown = widgets.Dropdown(
    options=['simple', 'normal', 'karras', 'exponential', 'sgm_uniform'],
    value='simple',
    description='Scheduler:',
    style=style,
    layout=layout
)

generate_btn = widgets.Button(
    description='🎨 Generate Image',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

output_area = widgets.Output()

def on_generate_click(b):
    with output_area:
        clear_output(wait=True)
        print("🎨 Generating image...")
        try:
            output_path, used_seed = generate(
                positive_prompt=prompt_input.value,
                negative_prompt=negative_input.value,
                width=width_slider.value,
                height=height_slider.value,
                steps=steps_slider.value,
                cfg=cfg_slider.value,
                seed=seed_input.value,
                sampler_name=sampler_dropdown.value,
                scheduler=scheduler_dropdown.value
            )
            clear_output(wait=True)
            print(f"✅ Image generated successfully!\n🎲 Seed used: {used_seed}")
            display(Image.open(output_path))
        except Exception as e:
            print(f"❌ Error: {str(e)}")

generate_btn.on_click(on_generate_click)

ui = widgets.VBox([
    widgets.HTML("<h2 style='color: #4CAF50;'>🎨 Z-Image Turbo Generator</h2>"),
    prompt_input,
    negative_input,
    widgets.HBox([width_slider, height_slider]),
    widgets.HBox([steps_slider, cfg_slider]),
    seed_input,
    sampler_dropdown,
    scheduler_dropdown,
    generate_btn,
    output_area
])

display(ui)